<a href="https://colab.research.google.com/github/sabaronett/REBOUNDxPaper/blob/master/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>REBOUNDx Interactive Demo<h1>

*This notebook is designed for [Google Colab](https://colab.research.google.com/github/sabaronett/REBOUNDxPaper/blob/master/colab_demo.ipynb); visit [Welcome To Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb) to learn more.*

Below are a few quick, interactive demos of REBOUNDx's new features and effects introduced in [*Baronet et al. (2022)*](https://doi.org/10.1093/mnras/stac043). You can find more resources and example notebooks at our paper's [GitHub repository](https://github.com/sabaronett/REBOUNDxPaper), and at [REBOUND](https://rebound.readthedocs.io/en/latest/) and [REBOUNDx](https://reboundx.readthedocs.io/en/latest/).

## Stellar Evolution

This example shows how to change a particle's mass by interpolating time-series data during a REBOUND simulation using REBOUNDx. Documentation for this feature can be found [here](https://reboundx.readthedocs.io/en/latest/effects.html#parameter-interpolation) and a more detailed example notebook [here](https://github.com/dtamayo/reboundx/blob/master/ipython_examples/ParameterInterpolation.ipynb).

First let's set up a simple Sun-Earth system. 

*To execute the code in the cell below, select it with a click and then either press the play button to the left of the code, or use the keyboard shortcut "Command/Ctrl+Enter". To edit the code, just click the cell and start editing. Anything you define in one cell can later be used in other cells.*

In [ ]:
!pip install -q rebound
import rebound

def makesim():
  sim = rebound.Simulation()        # create simulation object
  sim.units = ('yr', 'AU', 'Msun')  # set sim units
  sim.add(m=1.)                     # add Sun (1 Msun)
  sim.add(a=1.)                     # add Earth at 1 au
  sim.integrator = 'whfast'         # set integrator
  
  return sim

Next let's integrate this system as is for 10,000 years and verify Earth's semi-major axis does not change.

In [ ]:
%%time
import numpy as np
import matplotlib.pyplot as plt

sim = makesim()                     # initialize sim
ps = sim.particles                  # retrieve array of particle objects
Nout = 1000                         # set total # of outputs
times = np.linspace(0, 1e4, Nout)   # Nout evenly spaced intervals from 0 to 10K
a = np.zeros(Nout)                  # array for Earth's semi-major axis

for i, t in enumerate(times):       # loop through times
  sim.integrate(t)                  # advance sim to next output time
  # ps[0].m = starmass.interpolate(rebx, t) # interpolate & update Sun's mass
  a[i] = ps[1].a                    # store Earth's current semi-major axis

plt.plot(times, a)                  # plot Earth's semi-major axis vs. time
plt.xlabel('time / yr')
plt.ylabel('semi-major axis / au')
plt.ylim(0.5, 3.)
plt.show()

Now let's define a time series of the Sun's mass, consisting of just two equal-sized arrays. The following 5 values correspond to an exponential mass loss rate (e-folding timescale) of $10^4$ years. We then create an `Interpolator` object for the parameter set and pass our arrays as arguments. Before moving on, we plot the mass loss data for good measure.

In [ ]:
!pip install -q reboundx
import reboundx

mtimes = np.asarray([0, 2500, 5000, 7500, 10000])      # define time series
masses = np.asarray([1., 0.77880078307, 0.60653065971, # series of masses
                     0.47236655274, 0.36787944117])
rebx = reboundx.Extras(sim)                            # attach REBOUNDx to sim
starmass = reboundx.Interpolator(rebx, mtimes, masses, # create interp. obj.
                                 'spline')

plt.plot(mtimes, masses, 'o')                          # plot mass loss data
plt.xlabel('time / yr')
plt.ylabel('mass / Msun')
plt.show()

Finally, let's integrate the system again, but this time we update the Sun's mass by interpolating at the output intervals of the loop. To do this, we simply go back to our earlier code cell (two above), uncomment the following line:

```python
ps[0].m = starmass.interpolate(rebx, t) # interpolate & update Sun's mass
```

(the second line within the `for` loop), and rerun the cell. Notice the adiabatic expansion of the Earth this time.

## Tides

This example shows how to add a constant time lag model (Hut 1981) to tides raised on the primary and/or orbiting bodies using REBOUNDx. Documentation for this feature can be found [here](https://reboundx.readthedocs.io/en/latest/effects.html#tides) and a more detailed example notebook [here](https://github.com/dtamayo/reboundx/blob/master/ipython_examples/TidesConstantTimeLag.ipynb).

First we set up a simple Sun-Earth system, but this time we assume our Sun has lost some mass after leaving the main-sequence, and we give our Earth its proper mass and a slightly eccentric orbit.

In [ ]:
def makesim():
  sim = rebound.Simulation()
  sim.units = ('yr', 'AU', 'Msun')
  sim.add(m=0.86)                # post-MS Sun
  sim.add(m=3.e-6, a=1., e=0.03) # Earth w/ slight eccentricity
  sim.integrator = 'whfast'
  sim.move_to_com()              # recenter coordinates to center of mass
  rebx = reboundx.Extras(sim)    # initialize REBOUNDx
  tides = rebx.load_force("tides_constant_time_lag") # load TCTL
  rebx.add_force(tides)          # add tidal force
  
  return sim, rebx, tides

Next, we set the primary and secondaries' equilibrium gravitational response to the tidal field acting on them through the `tctl_k1`. If we give the primary a physical radius, then any (massive) orbiting body will raise equilibrium tides on the primary. If we add a physical radius and `tctl_k1` to any of the orbiting bodies, the primary will raise tides on that particle (but note that orbiting bodies will not raise tides on one another).

In [ ]:
sim, rebx, tides = makesim()
ps = sim.particles
ps[0].r = 0.85 # Sun's physical radius in au
ps[0].params["tctl_k1"] = 0.03

If we stop here and don't add a time lag, we will get the instantaneous equilibrium tide, which provides a conservative, radial non-Keplerian potential. The total energy will be conserved, but the pericenter will precess.

We integrate for 5000 years and plot Earth's longitude of the pericenter over time.

In [ ]:
%%time
tmax = 5000
times = np.linspace(0, tmax, Nout)
pomega = np.zeros(Nout)

for i, t in enumerate(times):
  sim.integrate(t)
  pomega[i] = ps[1].pomega          # store Earth's longitude of pericenter

plt.plot(times, pomega)             # plot Earth's pericenter precession
plt.xlabel('time / yr')
plt.ylabel('pericenter')
plt.show()

Setting the `tctl_tau` constant time lag parameter introduces dissipation, causing eccentricity damping, and will migrate the orbiting bodies either inward or outward depending on whether they orbit faster or slower than the spin of the tidally deformed body. We set the spin rate of each body with the `Omega` parameter. If it is not set, `Omega` is assumed to be zero.

Note the bodies' spins are assumed fixed, so consider whether more angular momentum is being changed in the system than is available in the spins! We additionally assume spins are aligned with the reference z axis.

In [ ]:
sim, rebx, tides = makesim()
ps = sim.particles
ps[0].r = 0.85                 # au
ps[0].params["tctl_k1"] = 0.03 # potential Love number, degree 2
ps[0].params["tctl_tau"] = 0.4 # small constant time lag
ps[0].params["Omega"] = 0      # 0 by default

Now we integrate this time for 25,000 years and plot Earth's semi-major axis over time.

In [ ]:
%%time
tmax = 2.5e4
times = np.linspace(0,tmax,Nout)
a, e = np.zeros(Nout), np.zeros(Nout)

for i, t in enumerate(times):
  sim.integrate(t)
  a[i] = ps[1].a        # store Earth's current semi-major axis
  e[i] = ps[1].e        # store Earth's current eccentricity

plt.plot(times, a)      # plot Earth's semi-major axis
plt.xlabel('time / yr')
plt.ylabel('semi-major axis')
plt.show()

Finally, we can also plot Earth's eccentricity over time.

In [ ]:
plt.plot(times, e)             # plot Earth's pericenter precession
plt.xlabel('time / yr')
plt.ylabel('eccentricity')
plt.show()